 ## Step 0

In [8]:
import pandas as pd
import numpy as np
import re

In [9]:
col_string = "C:F"
energy = pd.read_excel('data/Energy Indicators.xls',
                       usecols=col_string,
                       nrows = 227,
                       na_values=['...'],
                       skiprows = range(0,17))
energy.rename(columns = {'Unnamed: 2' : 'Country',
                         'Petajoules': 'Energy Supply',
                         'Gigajoules': 'Energy Supply per Capita',
                         '%': '% Renewable'}, inplace = True)

In [10]:
def changeCountry(country_name):
    if(re.search("^Republic of Korea",country_name)!=None):
        return "South Korea"
    
    elif(re.search("^United States of America",country_name)!=None):
        return "United States"
    
    elif(re.search("^United Kingdom of Great Britain and Northern Ireland",country_name)!=None):
        return "United Kingdom"
    
    elif(re.search("^China, Hong Kong Special Administrative Region",country_name)!=None):
        return "Hong Kong"
    return country_name

def removeParenthes(country_name):
    index= country_name.find('(')
    if(index != -1):
        return country_name[:index].strip()
    return country_name.strip()

def removeNumbers(country_name):
    country_name = re.sub("\d+", " ", country_name)
    return country_name.strip()

In [18]:

energy['Country'] = energy['Country'].apply(changeCountry).apply(removeParenthes).apply(removeNumbers)
energy.sample(10)

,Country,Energy Supply,Energy Supply per Capita,% Renewable
108,Kazakhstan,3548.0,208.0,8.112784
185,Slovenia,287.0,139.0,31.864240
187,Somalia,126.0,12.0,0.000000
177,Saudi Arabia,7985.0,264.0,0.000352
163,Qatar,1396.0,664.0,0.000000
43,Hong Kong,585.0,82.0,0.000000
82,Greece,993.0,90.0,24.795280
106,Jersey,3.0,34.0,0.000000
18,Belarus,1142.0,120.0,0.463389
200,Thailand,5336.0,79.0,4.305189



##  Step 1

In [19]:
GDP = pd.read_csv('data/world_bank.csv',
                       skiprows = range(0,4))

def changeCountryGDP(country_name):
    if(re.search("^Korea, Rep.",country_name)!=None):
        return "South Korea"
    
    elif(re.search("^Iran, Islamic Rep.",country_name)!=None):
        return "Iran"
    
    elif(re.search("^Hong Kong SAR, China",country_name)!=None):
        return "Hong Kong"
    
    return country_name
GDP['Country Name']= GDP['Country Name'].apply(changeCountryGDP)


### Step 2

In [20]:
ScimEn =pd.read_excel('data/scimagojr-3.xlsx',header=0)


### Step 3

In [21]:
new_df= pd.merge(energy,ScimEn[ScimEn['Rank'] <= 15],on='Country').rename(columns = {'Country' : 'Country Name'})
new_df= pd.merge(new_df,GDP,on='Country Name')

new_df2 = new_df.copy()

new_df =new_df.set_index('Country Name')
new_df = new_df[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
countriesRows = len(new_df['Rank'])

new_df


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country Name,,,,,,,,,,,,,,,,,,,,
Australia,14,18077,17661,263733,44799,14.59,161,5386.0,231.0,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,18024,17745,152380,37112,8.45,121,12149.0,59.0,69.648030,1.845080e+12,1.957118e+12,2.056809e+12,2.054215e+12,2.208872e+12,2.295245e+12,2.339209e+12,2.409740e+12,2.412231e+12,2.319423e+12
Canada,8,29633,29011,491467,88987,16.59,210,10431.0,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
China,1,235126,233883,1909601,1306438,8.12,224,127191.0,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
France,11,22429,21913,300015,58151,13.38,169,10597.0,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
Germany,6,32935,32227,367356,78265,11.15,186,13261.0,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,4,39893,38848,368175,123446,9.23,171,33195.0,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
Iran,12,19371,19085,242250,77012,12.51,127,9172.0,119.0,5.707721,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
Italy,9,23725,22819,312631,76446,13.18,155,6530.0,109.0,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12



### Step 4

In [22]:
orignal_df= pd.merge(energy,ScimEn,on='Country').rename(columns = {'Country' : 'Country Name'})
orignal_df= pd.merge(orignal_df,GDP,on='Country Name')
len(orignal_df['Country Name'])-countriesRows # number of lost rows

157


### Step 5

In [23]:
avgGDP =new_df.iloc[:, -10:-1].mean(axis=1) # mean for last 10 years(cols)
avgGDP

Country Name
Australia             1.148797e+12
Brazil                2.175391e+12
Canada                1.645985e+12
China                 6.076454e+12
France                2.672896e+12
Germany               3.471633e+12
India                 1.702863e+12
Iran                  4.441558e+11
Italy                 2.128048e+12
Japan                 5.528057e+12
South Korea           1.088952e+12
Russian Federation    1.559827e+12
Spain                 1.417885e+12
United Kingdom        2.468081e+12
United States         1.523276e+13
dtype: float64


### Step 6

In [24]:
(new_df[['Energy Supply per Capita']].mean(axis=0))[0]

157.6


### Step 7

In [25]:
maxRene = new_df.sort_values(by='% Renewable',ascending = False).head(1)

print ((maxRene.index[0], (maxRene[['% Renewable']].values[0])[0]))

('Brazil', 69.64803)



### Step 8

In [26]:
new_df['Citations Ratio']=  new_df['Self-citations'] / new_df['Citations']

maxRatio = new_df.sort_values(by='Citations Ratio',ascending = False).head(1)

print ((maxRatio.index[0], (maxRatio[['Citations Ratio']].values[0])[0]))


('China', 0.6841418704745127)



### Step 9

In [27]:
HighRenew = new_df.set_index('Rank')
median=(HighRenew[['% Renewable']].median(axis=0))[0]

def rankRenewable(renewable):
    if(renewable>=median):
        return "a 1"
    else:
        return "a 0"
HighRenew['Renewable Rank'] = HighRenew['% Renewable'].apply(rankRenewable)
HighRenew.sort_values(by='Rank')[['% Renewable','Renewable Rank']]


,% Renewable,Renewable Rank
Rank,,
1,19.754910,a 1
2,11.570980,a 0
3,10.232820,a 0
4,14.969080,a 0
5,10.600470,a 0
6,17.901530,a 1
7,17.288680,a 1
8,61.945430,a 1
9,33.667230,a 1




### Step 10

In [28]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
new_df2['Continent']=None

def getContinent(countryName):
    return ContinentDict.get(countryName, None)
new_df2['Continent'] = new_df2['Country Name'].apply(getContinent)

new_df2['population'] =pd.to_numeric(new_df2['Energy Supply'] / new_df2['Energy Supply per Capita'])

continent_df= new_df2[['Continent','Country Name','population']]
continent_df =continent_df.set_index('Continent')
continent_df = continent_df.groupby(continent_df.index)['population'].agg(['size', 'sum', 'mean', 'std'])
continent_df

,size,sum,mean,std
Continent,,,,
Asia,5,2898.666387,579.733277,679.097888
Australia,1,23.316017,23.316017,NaN
Europe,6,457.929667,76.321611,34.647667
North America,2,352.855249,176.427625,199.669645
South America,1,205.915254,205.915254,NaN
